In [4]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import cv2
import os
tf.config.run_functions_eagerly(True)


# Set the seed for reproducibility
tf.random.set_seed(42)

# Set the paths for the training data
data_folder_path = r"C:\Users\Nitro5\OneDrive - АО Казахстанско-Британский Технический Университет\Рабочий стол\comp and deep"
subjects = ["", "Ernar Naryshov", "D Trump or Tramp"]

# Define a function to preprocess the images
def preprocess_images(img):
    img = cv2.resize(img, (128, 128))
    img = img / 255.0
    return img

# Define a function to prepare the training data
def prepare_training_data(data_folder_path):
    dirs = os.listdir(data_folder_path)
    images = []
    labels = []
    for dir_name in dirs:
        if not dir_name.startswith("s"):
            continue
        label = int(dir_name.replace("s", ""))
        subject_dir_path = data_folder_path + "/" + dir_name
        subject_images_names = os.listdir(subject_dir_path)
        for image_name in subject_images_names:
            if image_name.startswith("."):
                continue
            image_path = subject_dir_path + "/" + image_name
            image = cv2.imread(image_path)
            if image is not None:
                image = preprocess_images(image)
                images.append(image)
                labels.append(label)
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

# Prepare the training data
X_train, y_train = prepare_training_data(data_folder_path)

# Define the model architecture
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])

# Compile the model
# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'], run_eagerly=True)
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], run_eagerly=True)


# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Load a test image
test_image = cv2.imread(r"C:\Users\Nitro5\OneDrive - АО Казахстанско-Британский Технический Университет\Рисунки\Пленка\WIN_20230329_00_16_53_Pro")
test_image = preprocess_images(test_image)
test_image = np.expand_dims(test_image, axis=0)

# Use the model to predict the label of the test image
prediction = model.predict(test_image)
predicted_label = subjects[np.argmax(prediction)]

# Print the predicted label
print(predicted_label)


Epoch 1/10


ValueError: Unexpected result of `train_function` (Empty logs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.